# Wine Quality — Polynomial Linear Regression (one command per code cell)
# Steps: load → checks → split → pipeline(PolynomialFeatures+LinearRegression) → evaluate → CV → persistence (commented) → inference.


# Imports

In [ ]:
import pandas as pd  # data handling


In [ ]:
import numpy as np  # numeric helpers


In [ ]:
from sklearn.model_selection import train_test_split, cross_validate  # split + CV


In [ ]:
from sklearn.preprocessing import PolynomialFeatures  # polynomial feature generator


In [ ]:
from sklearn.linear_model import LinearRegression  # linear regressor (used after poly expansion)


In [ ]:
from sklearn.pipeline import Pipeline  # chain preprocessing and model


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # metrics


# Load dataset selector

In [ ]:
DATASET = 'red'  # choose 'red' or 'white'


In [ ]:
CSV_PATH = f"/Users/sb/Documents/Study-Material/Machine-Learning/winequality-{DATASET}.csv"  # dataset path


In [ ]:
df = pd.read_csv(CSV_PATH, sep=';')  # read CSV with semicolon separator


# Split

In [ ]:
X = df.drop(columns=['quality'])  # features


In [ ]:
y = df['quality']  # target


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # hold-out split


# Pipeline: PolynomialFeatures + LinearRegression

In [ ]:
poly_deg = 2  # degree of polynomial features (change to 3 for higher-order)


In [ ]:
pipe = Pipeline([('poly', PolynomialFeatures(degree=poly_deg, include_bias=False)), ('lin', LinearRegression())])  # build pipeline


In [ ]:
pipe.fit(X_train, y_train)  # fit pipeline on training data


In [ ]:
pred = pipe.predict(X_test)  # predict on test set


# Evaluate

In [ ]:
mae = mean_absolute_error(y_test, pred)  # MAE


In [ ]:
rmse = mean_squared_error(y_test, pred, squared=False)  # RMSE


In [ ]:
r2 = r2_score(y_test, pred)  # R^2


In [ ]:
{'MAE': mae, 'RMSE': rmse, 'R2': r2}  # metrics summary


# Cross-validation

In [ ]:
cv = cross_validate(pipe, X, y, cv=5, scoring=('r2','neg_root_mean_squared_error'))  # 5-fold CV


In [ ]:
{k: (np.mean(v), np.std(v)) for k, v in cv.items() if k.startswith('test_')}  # mean±std of CV scores


# MLflow Tracking — log params, metrics, and model

In [ ]:
import mlflow  # experiment tracking


In [ ]:
import mlflow.sklearn  # sklearn flavor


In [ ]:
mlflow.set_experiment('PolynomialRegression-WineQuality')  # experiment per algorithm


In [ ]:
mlflow.start_run(run_name=f"{DATASET}")  # start run


In [ ]:
mlflow.log_params({'degree': poly_deg})  # log polynomial degree


In [ ]:
mlflow.log_metric('MAE', mae)  # MAE


In [ ]:
mlflow.log_metric('RMSE', rmse)  # RMSE


In [ ]:
mlflow.log_metric('R2', r2)  # R2


In [ ]:
mlflow.sklearn.log_model(pipe, artifact_path='model')  # log pipeline


In [ ]:
mlflow.end_run()  # end run


# Persistence — save to disk and reload

In [ ]:
import os  # fs utils


In [ ]:
import joblib  # persistence


In [ ]:
from datetime import datetime  # timestamp


In [ ]:
os.makedirs('Machine-Learning/models/PolynomialRegression', exist_ok=True)  # ensure dir


In [ ]:
MODEL_PATH = f"Machine-Learning/models/PolynomialRegression/polynomial_regression_deg{poly_deg}_wine_{DATASET}_" + datetime.now().strftime('%Y%m%d_%H%M%S') + ".joblib"  # path


In [ ]:
joblib.dump(pipe, MODEL_PATH)  # save pipeline


In [ ]:
loaded = joblib.load(MODEL_PATH)  # load back


In [ ]:
float(loaded.predict(X_test.iloc[[0]])[0])  # quick check


# Streamlit App — minimal predictor UI

In [ ]:
APP_CODE = """
import joblib, pandas as pd, streamlit as st
st.set_page_config(page_title='Wine — Polynomial Regression', page_icon='🍷')
st.title('🍷 Wine Quality — Polynomial Regression')
MODEL_PATH = st.text_input('Model path', value=r'%s')
if MODEL_PATH and st.button('Load model'):
    try:
        st.session_state['model'] = joblib.load(MODEL_PATH)
        st.success('Model loaded')
    except Exception as e:
        st.error(f'Load failed: {e}')
if 'model' in st.session_state:
    cols = %s
    vals = {c: st.number_input(c, value=0.0) for c in cols}
    if st.button('Predict quality'):
        df = pd.DataFrame([vals])
        try:
            y = st.session_state['model'].predict(df)
            st.metric('Predicted quality', f"{float(y[0]):.3f}")
        except Exception as e:
            st.error(f'Predict failed: {e}')
""" % (MODEL_PATH, list(X.columns))


In [ ]:
open('app_streamlit_wine.py', 'w').write(APP_CODE)  # write app file


# AWS S3 Upload (commented)

In [ ]:
# import boto3


In [ ]:
# s3 = boto3.client('s3')


In [ ]:
# bucket = os.environ.get('S3_BUCKET','your-bucket')


In [ ]:
# key = 'wine-models/PolynomialRegression/' + os.path.basename(MODEL_PATH)


In [ ]:
# s3.upload_file(MODEL_PATH, bucket, key)


In [ ]:
float(pipe.predict(X_test.iloc[[0]])[0])  # single-row prediction via pipeline
